In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *

In [2]:
steps = 5000
segs = 20 # number of segments
Paths = np.load("Paths_{}.npy".format(steps))
n_paths = Paths.shape[0]

In [3]:
x0 = 1.0 # initial condition
sigma = 1 # volatility
r = 0.01 # risk free rate
batch_size = 1000 # batch size
T = 1 # maturity
dt = T/steps # mesh size

In [4]:
shrink = int(steps/segs) # eg. shrink steps to segments
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level
sig_dim = iisig.siglength(2,level)# dimension of siganature
logsig_dim = iisig.logsiglength(2,level)# dimension of log-siganature

In [5]:
def generate_t(T, batch_size = batch_size):
    '''
    generate time steps
    dim = batch_size * (steps+1). eg. 2000 * 101
    '''
    return np.tile(np.linspace(1e-8, T+1e-8, steps + 1, dtype = np.float32), (batch_size,1))

In [6]:
t = generate_t(T, batch_size)[0][0:]
len(t)

101

In [7]:
# Create Sig and Log-Sig from Paths.

In [8]:
def Create_Signatures(paths, segments = segs):
    '''
    input: paths & # of segmentations
    output: (1) log-sigatures: 
                dim = (n_paths * (segments+1) * logsig_dim), 
            (2) sigatures: 
                dim = (n_paths * (segments+1) * sig_dim),
    '''

    # Initialization
    n_paths = paths.shape[0]
    logsig = np.zeros((n_paths, segments + 1, logsig_dim))
    sig = np.zeros((n_paths, segments + 1, sig_dim))
    t = generate_t(T, steps)[0, None, :]
    
    # signature at time 0 = x_0
    path_cut = paths[:,0] # cut paths into segments
    time_cut = t[:, 0]  # cut time into segments
    s = iisig.prepare(2, level)
    for i in range(n_paths): 
        stream = np.hstack((time_cut.reshape((-1,1)), path_cut[i].reshape((-1,1))))  #
        sig[i, 0, :] = iisig.sig(stream, level) # dim = (84, )
        logsig[i, 0, :] = iisig.sig(stream, level)[:logsig_dim]

    # This step computes signatures.
    for k in tqdm(range(segments)):
        path_cut = paths[:, (k*shrink):(k+1)*shrink+1]
        time_cut = t[:, (k*shrink):(k+1)*shrink+1]
        for i in range(n_paths):
            stream = np.hstack((time_cut.reshape((-1,1)), path_cut[i].reshape((-1,1))))
            sig[i, k+1, :] = iisig.sig(stream, level)
            logsig[i, k+1, :] = iisig.logsig(stream, s)

            
    return {'logsig': logsig, 'sig': sig}

In [9]:
print("Creating Sig and Log-Sig with {} segments from Paths with {} steps... ".format(segs, steps))
logsig_sig = Create_Signatures(Paths, segments = segs)
logsig = logsig_sig["logsig"]
sig = logsig_sig["sig"]

Creating Sig and Log-Sig with 5 segments from Paths with 100 steps... 


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.01it/s]


In [10]:
np.save("logsig_{}_from_{}.npy".format(segs, steps), logsig)
np.save("sig_{}_from_{}.npy".format(segs, steps), sig)

In [11]:
# Selection of W 
W_full= np.load("W_{}.npy".format(steps))
selection = np.linspace(0,steps, segs+1, dtype = np.int)
W_segs = W_full[:,selection]

In [12]:
np.save("W_{}_from_{}.npy".format(segs, steps), W_segs)